In [29]:
# import required modules
import random
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import model_selection, metrics, preprocessing
import copy

import torch
from torch import nn, optim, Tensor

from torch_sparse import SparseTensor, matmul

from collections import defaultdict

from torch_geometric.utils import structured_negative_sampling
from torch_geometric.data import download_url, extract_zip
from torch_geometric.nn.conv.gcn_conv import gcn_norm
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.typing import Adj
import torch.nn.functional as F

In [30]:
rating_df = pd.read_csv('ratings.csv')


In [31]:
rating_df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [32]:
rating_df.describe()

,userId,movieId,rating,timestamp
count,100836.000000,100836.000000,100836.000000,1.008360e+05
mean,326.127564,19435.295718,3.501557,1.205946e+09
std,182.618491,35530.987199,1.042529,2.162610e+08
min,1.000000,1.000000,0.500000,8.281246e+08
25%,177.000000,1199.000000,3.000000,1.019124e+09
50%,325.000000,2991.000000,3.500000,1.186087e+09
75%,477.000000,8122.000000,4.000000,1.435994e+09
max,610.000000,193609.000000,5.000000,1.537799e+09


In [33]:
rating_df.describe()

,userId,movieId,rating,timestamp
count,100836.000000,100836.000000,100836.000000,1.008360e+05
mean,326.127564,19435.295718,3.501557,1.205946e+09
std,182.618491,35530.987199,1.042529,2.162610e+08
min,1.000000,1.000000,0.500000,8.281246e+08
25%,177.000000,1199.000000,3.000000,1.019124e+09
50%,325.000000,2991.000000,3.500000,1.186087e+09
75%,477.000000,8122.000000,4.000000,1.435994e+09
max,610.000000,193609.000000,5.000000,1.537799e+09


In [34]:
# label encoding using LabelEncoder
from sklearn.preprocessing import LabelEncoder

lbl_user = LabelEncoder()
lbl_movie = LabelEncoder()

rating_df['userId'] = lbl_user.fit_transform(rating_df['userId'].values)
rating_df['movieId'] = lbl_movie.fit_transform(rating_df['movieId'].values)

In [35]:
rating_df.describe()

,userId,movieId,rating,timestamp
count,100836.000000,100836.000000,100836.000000,1.008360e+05
mean,325.127564,3101.735561,3.501557,1.205946e+09
std,182.618491,2627.050983,1.042529,2.162610e+08
min,0.000000,0.000000,0.500000,8.281246e+08
25%,176.000000,900.000000,3.000000,1.019124e+09
50%,324.000000,2252.000000,3.500000,1.186087e+09
75%,476.000000,5095.250000,4.000000,1.435994e+09
max,609.000000,9723.000000,5.000000,1.537799e+09


In [36]:
def edger(df,src,dest,link,rating_threshold=3):
    edge_index=None
    source=[user_id for user_id in df["userId"]]
    destination=[movie_id for movie_id in df["movieId"]]

    # apply rating threshold
    edge_attr=torch.from_numpy(df[link].values).view(-1,1).to(torch.long)>=rating_threshold

    edge_index=[[],[]]

    for i in range(edge_attr.shape[0]):
        if edge_attr[i]:
            edge_index[0].append(source[i])
            edge_index[1].append(destination[i])
    


    return edge_index
    

In [37]:
edge_index=edger(rating_df,"userId","movieId","rating",3.5);

In [38]:
print(len(edge_index[0]))

48580


In [39]:
# convert to tensor
edge_index=torch.LongTensor(edge_index)
print(edge_index)
print(edge_index.shape)

tensor([[   0,    0,    0,  ...,  609,  609,  609],
        [   0,    2,    5,  ..., 9443, 9444, 9445]])
torch.Size([2, 48580])


In [40]:
# original unique number of users,movies
num_users=len(rating_df['userId'].unique())
num_movies=len(rating_df['movieId'].unique())

In [41]:
num_interactions=edge_index.shape[1];

# train test validation 80/10/10
all_indices=[i for i in range(num_interactions)]

train_indices, test_indices = train_test_split(all_indices, test_size=0.2, random_state=42)
test_indices, val_indices = train_test_split(test_indices, test_size=0.5, random_state=42)

train_edge_index=edge_index[:,train_indices]
val_edge_index=edge_index[:,val_indices]
test_edge_index=edge_index[:,test_indices]


In [42]:
def convert_r_mat_edge_index_to_adj_mat_edge_index( input_edge_index):
    R=torch.zeros((num_users,num_movies))
    for i in range(len(input_edge_index[0])):
        R[input_edge_index[0,i],input_edge_index[1,i]]=1
    
    R.transpose=torch.transpose(R,0,1)
    adj_mat=torch.zeros((num_users+num_movies,num_users+num_movies))
    adj_mat[:num_users,num_users:]=R.clone()
    adj_mat[num_users:,:num_users]=R.transpose.clone()
    adj_mat_coo=adj_mat.to_sparse_coo()
    adj_mat_coo=adj_mat_coo.indices()

    return adj_mat_coo

In [43]:
def convert_adj_mat_edge_index_to_r_mat_edge_index(input_edge_index):
    sparse_input_edge_index= torch.sparse_coo_tensor(input_edge_index[0],
                                                     input_edge_index[1],
                                                     sparse_sizes=((num_users+num_users),(num_users+num_users)))
    
    adj_mat = sparse_input_edge_index.to_dense()
    interact_mat = adj_mat[:num_users,num_users:]
    r_mat_edge_index=interact_mat.to_sparse_coo().indices()

    return r_mat_edge_index

In [44]:
# convert from r_mat (interaction matrix) to adj_mat (adjacency matrix) 
train_edge_index =convert_r_mat_edge_index_to_adj_mat_edge_index(train_edge_index)
val_edge_index =convert_r_mat_edge_index_to_adj_mat_edge_index(val_edge_index)
test_edge_index = convert_r_mat_edge_index_to_adj_mat_edge_index(test_edge_index)

In [45]:
print(train_edge_index)
print (train_edge_index.size())
print(val_edge_index)
print(val_edge_index.size())
print(test_edge_index)
print(test_edge_index.size())


tensor([[    0,     0,     0,  ..., 10327, 10329, 10333],
        [  610,   615,   653,  ...,   183,   183,   330]])
torch.Size([2, 77728])
tensor([[    0,     0,     0,  ..., 10290, 10294, 10305],
        [  656,   734,  1344,  ...,   595,    61,   379]])
torch.Size([2, 9716])
tensor([[    0,     0,     0,  ..., 10299, 10301, 10309],
        [  612,  1392,  1429,  ...,   555,   248,   317]])
torch.Size([2, 9716])


In [50]:
def sample_mini_batch(batch_size,edge_index):
    edges= structured_negative_sampling(edge_index)
    edges= torch.stack(edges,dim=0)

    indices= torch.choices([i for i in range(edges[0].shape[0])],k=batch_size)
    batch = edges[:,indices]

    user_indices,pos_item_indices,neg_item_indices=batch[0],batch[1],batch[2]
    return user_indices,pos_item_indices,neg_item_indices

In [51]:
# print ratings df
rating_df.head()

,userId,movieId,rating,timestamp
0,0,0,4.0,964982703
1,0,2,4.0,964981247
2,0,5,4.0,964982224
3,0,43,5.0,964983815
4,0,46,5.0,964982931


In [52]:
from torch_geometric.nn.conv import MessagePassing

In [55]:
# defines LightGCN model
class LightGCN(MessagePassing):
    """LightGCN Model as proposed in https://arxiv.org/abs/2002.02126
    """

    def __init__(self, num_users, num_items, embedding_dim=64, K=3, add_self_loops=False):

        super().__init__()
      
        self.num_users = num_users
        self.num_items = num_items
        self.embedding_dim = embedding_dim
        self.K = K
        self.add_self_loops = add_self_loops



        self.users_emb = nn.Embedding(num_embeddings=self.num_users, embedding_dim=self.embedding_dim) # e_u^0
        
        self.items_emb = nn.Embedding(num_embeddings=self.num_items, embedding_dim=self.embedding_dim) # e_i^0

        #better performance according to the paper (LightGCN)
        nn.init.normal_(self.users_emb.weight, std=0.1)
        nn.init.normal_(self.items_emb.weight, std=0.1)
        

        # self.out = nn.Linear(embedding_dim + embedding_dim, 1)

    def forward(self, edge_index: Tensor, edge_values: Tensor):
        #   def compute_gcn_norm(edge_index, emb):
        #             emb = emb.weight
        #             from_, to_ = edge_index
        #             deg = degree(to_, emb.size(0), dtype=emb.dtype)
        #             deg_inv_sqrt = deg.pow(-0.5)
        #             deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0
        #             norm = deg_inv_sqrt[from_] * deg_inv_sqrt[to_]

        #             return norm
        
        edge_index_norm = gcn_norm(edge_index=edge_index, 
                                   add_self_loops=self.add_self_loops)


        # concatenating the user and item embeddings ,total shape: (num_users + num_items) x embedding_dim
        emb_0 = torch.cat([self.users_emb.weight, self.items_emb.weight]) 

        embs = [emb_0] # layer 0 embeddings
        
        # 
        emb_k = emb_0 


        for i in range(self.K):
            emb_k = self.propagate(edge_index=edge_index_norm[0], x=emb_k, norm=edge_index_norm[1])
            embs.append(emb_k)
            
        # shape: (num_users + num_items) x (K + 1) x embedding_dim
        embs = torch.stack(embs, dim=1)
        
        # better performance according to the paper (LightGCN)
        emb_final = torch.mean(embs, dim=1) 
        
        users_emb_final, items_emb_final = torch.split(emb_final, 
                                                       [self.num_users, self.num_items]) # splits into e_u^K and e_i^K


        r_mat_edge_index, _ = convert_adj_mat_edge_index_to_r_mat_edge_index(edge_index, edge_values)
        
        src, dest =  r_mat_edge_index[0], r_mat_edge_index[1]
        
        # applying embedding lookup to get embeddings for src nodes and dest nodes in the edge list
        user_embeds = users_emb_final[src]
        item_embeds = items_emb_final[dest]
        
        # output dim: edge_index_len x 128 (given 64 is the original emb_vector_len)
        output = torch.cat([user_embeds, item_embeds], dim=1)
        
        # push it through the linear layer
        output = self.out(output)
        
        return output
    
    def message(self, x_j, norm):
        return norm.view(-1, 1) * x_j

layers = 3
model = LightGCN(num_users=num_users, 
                 num_items=num_movies, 
                 K=layers)

In [1]:
import torch.nn.functional as F
from torch.optim import Adam

In [ ]:
loss_fn = nn.BCEWithLogitsLoss()

In [ ]:
def bprloss(user_emb_final,user_emb_0,pos_items_emb_final,pos_items_emb_0,neg_items_emb_final,neg_items_emb_0,lambda_val):
    reg_loss =lambda_val*(user_emb_0.norm2(2).pow(2)+pos_items_emb_0.norm(2).pow(2)+neg_items_emb_0.norm(2).pow(2))
    #L2 loss

    pos_scores = torch.mul(user_emb_final,pos_items_emb_final)
    pos_scores = torch.sum(pos_scores,dim=-1)
    neg_scores = torch.mul(user_emb_final,neg_items_emb_final)
    neg_scores = torch.sum(neg_scores,dim=-1)

    bprloss = -torch.mean(torch.nn.functional.softplus(pos_scores-neg_scores))

    loss=reg_loss+bprloss
    return loss